
### Requirements

In [1]:
import numpy as np
import pandas as pd
import nltk
import pickle
from nltk.util import bigrams
import geopandas as gpd
import warnings
warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [100]:
# requires to be connected to a properly configured python environment with tensorflow etc.
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from tensorflow import keras
from keras import models
from keras import layers


In [2]:
try:
    LIREg = gpd.read_parquet("../data/large_data/LIREg.parquet")
    print("read locally")
except:
    LIREg = gpd.read_file("https://zenodo.org/record/5074774/files/LIREg.geojson?download=1", driver="geoJSON")
    LIREg.to_parquet("../data/large_data/LIREg.parquet")
    print("locally not available yet, need to download")
LIREg.head()

read locally


,EDCS-ID,publication,province,province_list,place,place_list,end_yr_list,notes_dating,status_list,inscr_type,...,within_rome,nearest_city,city_id_hanson,city_pop_est,city_geometry,nearest_city_type,nearest_city_dist,type_of_inscription_auto,type_of_inscription_auto_prob,geometry
0,EDCS-03700724,"ZPE-108-159 = Thesprotia 00001 = AE 1993, 0140...",Achaia,Achaia,Agios Athanasios / Photike,"['Agios Athanasios', 'Photike']",313,,"['Augusti/Augustae', 'litterae erasae', 'ordo ...",tituli honorarii,...,False,Dodona,31,1000.0,"[20.787767, 39.546432]",minor,0.097513,honorific inscription,1.0,POINT (20.76680 39.45120)
1,EDCS-03300852,"AE 1995, 01409",Achaia,Achaia,Alea / Tegea,"['Alea', 'Tegea']",276,to 276; b: 276 to 282 \n\n,"['Augusti/Augustae', 'miliaria', 'viri']",miliaria,...,False,Tegea,97,46362.0,"[22.417226, 37.427653]",big,0.004249,mile-/leaguestone,1.0,POINT (22.41710 37.43190)
2,EDCS-28500283,"CIL 03, 07251 = D 00214 = NDIEC-07, p 81 = AE ...",Achaia,Achaia,Alea / Tegea,"['Alea', 'Tegea']",50,,"['Augusti/Augustae', 'leges', 'viri']",leges,...,False,Tegea,97,46362.0,"[22.417226, 37.427653]",big,0.004249,public legal inscription,1.0,POINT (22.41710 37.43190)
3,EDCS-09400671,"CIMRM-02, 02350 = IG-12, 00274 = Andros 00124 ...",Achaia,Achaia,Andros,Andros,209,,"['litterae erasae', 'tituli sacri']",tituli sacri,...,False,Ioulis,47,1000.0,"[24.34625, 37.633122]",minor,0.520308,votive inscription,1.0,POINT (24.83230 37.81880)
4,EDCS-24600769,"AE 1995, 01407 = AE 2001, 01812",Achaia,Achaia,Archea Olimpia / Archaia Olympia / Olympia,"['Archea Olimpia', 'Archaia Olympia', 'Olympia']",96,,{ },{ },...,False,Elis,35,1000.0,"[21.435443, 37.827452]",minor,0.262624,owner/artist inscription,1.0,POINT (21.62710 37.64790)


In [3]:
EDH_overlap_all = LIREg[(LIREg["EDH-ID"].notnull()) & (LIREg["EDCS-ID"].notnull())]
EDH_overlap = EDH_overlap_all[~EDH_overlap_all["type_of_inscription_clean"].str.contains("NULL")]
EDH_overlap.groupby("type_of_inscription_clean").size().sort_values(ascending=False)

type_of_inscription_clean
epitaph                            21520
votive inscription                 11728
owner/artist inscription            3340
honorific inscription               3003
building/dedicatory inscription     2561
mile-/leaguestone                   1307
identification inscription           850
acclamation                          287
defixio                              269
list                                 246
military diploma                     209
label                                194
boundary inscription                 175
elogium                              132
letter                               119
public legal inscription             109
seat inscription                      42
private legal inscription             36
prayer                                18
assignation inscription               15
calendar                              10
adnuntiatio                            1
dtype: int64

In [4]:
len(EDH_overlap.groupby("type_of_inscription_clean").size().sort_values(ascending=False))

22

In [ ]:
classes_sorted = [key for key in dict(EDH_overlap.groupby("type_of_inscription_clean").size().sort_values(ascending=False))]

In [ ]:
EDH_sampled = EDH_overlap # (might be constrained by an index, if needed)

### Labels (y) to one-hot-encoding format

In [13]:
y = EDH_sampled["type_of_inscription_clean"].tolist()
y[:10]

['honorific inscription',
 'mile-/leaguestone',
 'public legal inscription',
 'votive inscription',
 'owner/artist inscription',
 'public legal inscription',
 'honorific inscription',
 'building/dedicatory inscription',
 'building/dedicatory inscription',
 'building/dedicatory inscription']

In [14]:
set(y)

{'acclamation',
 'adnuntiatio',
 'assignation inscription',
 'boundary inscription',
 'building/dedicatory inscription',
 'calendar',
 'defixio',
 'elogium',
 'epitaph',
 'honorific inscription',
 'identification inscription',
 'label',
 'letter',
 'list',
 'mile-/leaguestone',
 'military diploma',
 'owner/artist inscription',
 'prayer',
 'private legal inscription',
 'public legal inscription',
 'seat inscription',
 'votive inscription'}

In [15]:
len(y)

46171

In [16]:
labels_dict = {}
for i, label in enumerate(list(set(y))):
    labels_dict[label] = i

In [144]:
labels_dict

{'military diploma': 0,
 'identification inscription': 1,
 'letter': 2,
 'seat inscription': 3,
 'building/dedicatory inscription': 4,
 'epitaph': 5,
 'elogium': 6,
 'public legal inscription': 7,
 'acclamation': 8,
 'votive inscription': 9,
 'calendar': 10,
 'assignation inscription': 11,
 'honorific inscription': 12,
 'label': 13,
 'mile-/leaguestone': 14,
 'boundary inscription': 15,
 'defixio': 16,
 'prayer': 17,
 'private legal inscription': 18,
 'list': 19,
 'adnuntiatio': 20,
 'owner/artist inscription': 21}

In [142]:
with open("../data/labels_dict.pickle", "wb") as f:
    pickle.dump(labels_dict, f)

In [17]:
# y
y = [labels_dict[label] for label in y]
y# labels to integers (otherwise the code below does not work...)

[12,
 14,
 7,
 9,
 21,
 7,
 12,
 4,
 4,
 4,
 1,
 4,
 5,
 9,
 5,
 5,
 5,
 5,
 5,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 1,
 5,
 12,
 4,
 12,
 5,
 5,
 12,
 5,
 5,
 12,
 6,
 12,
 12,
 12,
 5,
 5,
 9,
 12,
 5,
 5,
 7,
 5,
 12,
 12,
 12,
 12,
 12,
 5,
 12,
 12,
 5,
 12,
 9,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 9,
 5,
 5,
 12,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 12,
 12,
 12,
 12,
 12,
 4,
 4,
 4,
 12,
 9,
 4,
 5,
 7,
 4,
 7,
 7,
 12,
 5,
 4,
 14,
 5,
 12,
 9,
 14,
 9,
 12,
 5,
 5,
 5,
 9,
 21,
 9,
 9,
 7,
 5,
 8,
 5,
 21,
 5,
 5,
 5,
 21,
 12,
 12,
 12,
 4,
 4,
 14,
 9,
 12,
 12,
 9,
 9,
 21,
 5,
 12,
 5,
 5,
 5,
 7,
 5,
 4,
 5,
 5,
 4,
 7,
 5,
 4,
 5,
 21,
 7,
 9,
 4,
 12,
 9,
 4,
 12,
 14,
 7,
 5,
 9,
 5,
 5,
 7,
 0,
 12,
 4,
 9,
 9,
 5,
 1,
 21,
 21,
 21,
 5,
 1,
 9,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 4,
 9,
 9,
 5,
 5,
 5,
 5,
 14,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 21,
 1,
 5,
 5,
 5,
 9,
 5,
 5,
 14,
 5,
 5,

In [18]:
def to_one_hot(y, dimension=22):
    results = np.zeros((len(y), dimension))
    for i, label in enumerate(y):
        results[i, label] = 1.
    return results

one_hot_labels = to_one_hot(y)

In [19]:
one_hot_labels[:10]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,

In [25]:
with open("../data/large_data/one_hot_labels.pickle", "wb") as f:
    pickle.dump(one_hot_labels, f, protocol=pickle.HIGHEST_PROTOCOL)

### From attributes to features matrix

In [26]:
inscr_texts = EDH_sampled["clean_text_interpretive_word"].tolist()

In [27]:
def get_bigrams_underscore(inscr_text):
    try:
        inscr_bigrams = [" ".join(el) for el in list(bigrams(inscr_text.split()))]
        inscr_bigrams_ = [bigram.replace(" ", "_") for bigram in inscr_bigrams]
    except:
        inscr_bigrams_ = []
    return inscr_bigrams_

In [28]:
EDH_bigrams_ = [get_bigrams_underscore(inscr_text) for inscr_text in inscr_texts]
EDH_bigrams_[:5]

[['Fortissimo_et',
  'et_piissimo',
  'piissimo_Caesari',
  'Caesari_domino',
  'domino_nostro',
  'nostro_Galerio',
  'Galerio_Valerio',
  'Valerio_Maximiano',
  'Maximiano_Pio',
  'Pio_Felici',
  'Felici_Invicto',
  'Invicto_Coranius',
  'Coranius_Titianus',
  'Titianus_vir',
  'vir_perfectissimus',
  'perfectissimus_praeses',
  'praeses_provinciae',
  'provinciae_veteris',
  'veteris_Epiri',
  'Epiri_numini',
  'numini_eorum',
  'eorum_dicatissimus'],
 ['Imperatori_Caesari',
  'Caesari_Marco',
  'Marco_Annio',
  'Annio_Floriano',
  'Floriano_Pio',
  'Pio_Felici',
  'Felici_Augusto',
  'Augusto_patri',
  'patri_patriae',
  'patriae_milia',
  'milia_passuum',
  'passuum_III',
  'III_Imperatori',
  'Imperatori_Caesari',
  'Caesari_Marco',
  'Marco_Aurelio',
  'Aurelio_Probo',
  'Probo_Pio',
  'Pio_Felici',
  'Felici_Augusto',
  'Augusto_milia',
  'milia_passuum',
  'passuum_II'],
 ['Tiberius_Claudius',
  'Claudius_Caesar',
  'Caesar_Augustus',
  'Augustus_Germanicus',
  'Germanicus_pon

In [29]:
# flat list of all bigrams
bigrams_list = [el for sublist in EDH_bigrams_ for el in sublist]
bigrams_list[:10]

['Fortissimo_et',
 'et_piissimo',
 'piissimo_Caesari',
 'Caesari_domino',
 'domino_nostro',
 'nostro_Galerio',
 'Galerio_Valerio',
 'Valerio_Maximiano',
 'Maximiano_Pio',
 'Pio_Felici']

In [30]:
# ok, let's list only N of the most common
bigrams_mostfreq = nltk.FreqDist(bigrams_list).most_common()
bigrams_mostfreq = [tup[0] for tup in bigrams_mostfreq]
bigrams_mostfreq[:10]

['Dis_Manibus',
 'vixit_annos',
 'votum_solvit',
 'solvit_libens',
 'libens_merito',
 'Iovi_Optimo',
 'Optimo_Maximo',
 'hic_situs',
 'situs_est',
 'tribunicia_potestate']

In [31]:
all_bigrams_freq_dict = dict(nltk.FreqDist(bigrams_list).most_common())
EDH_sampled["bigrams"] = EDH_bigrams_
all_bigrams_freq_dict


/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


{'Dis_Manibus': 7844,
 'vixit_annos': 4084,
 'votum_solvit': 3335,
 'solvit_libens': 2982,
 'libens_merito': 2710,
 'Iovi_Optimo': 2413,
 'Optimo_Maximo': 2399,
 'hic_situs': 2338,
 'situs_est': 2228,
 'tribunicia_potestate': 1990,
 'Manibus_sacrum': 1865,
 'bene_merenti': 1585,
 'sibi_et': 1476,
 'Imperatori_Caesari': 1256,
 'hic_sita': 1216,
 'sita_est': 1155,
 'faciendum_curavit': 1118,
 'pro_salute': 1117,
 'terra_levis': 1111,
 'Imperator_Caesar': 1099,
 'tibi_terra': 1029,
 'sit_tibi': 999,
 'vixit_annis': 996,
 'pro_praetore': 932,
 'pontifex_maximus': 837,
 'miles_legionis': 783,
 'pontifici_maximo': 780,
 'est_sit': 749,
 'maximus_tribunicia': 708,
 'maximo_tribunicia': 706,
 'ex_voto': 701,
 'Pio_Felici': 698,
 'pater_patriae': 698,
 'patri_patriae': 687,
 'divi_Nervae': 670,
 'et_suis': 664,
 'et_I': 663,
 'Marcus_Aurelius': 656,
 'qui_vixit': 634,
 'piae_fidelis': 611,
 'Caius_Iulius': 608,
 'Marco_Aurelio': 600,
 'libens_laetus': 588,
 'decreto_decurionum': 585,
 'divi_Tra

In [66]:
N = 500
weighted_bigrams_all = []
for cat in EDH_sampled["type_of_inscription_clean"].unique():
    EDH_cat = EDH_sampled[EDH_sampled["type_of_inscription_clean"]==cat]
    bigrams_list = [el for sublist in EDH_cat["bigrams"].tolist() for el in sublist]
    cat_bigrams_freq_dict = dict(nltk.FreqDist(bigrams_list).most_common())
    weighted_bigrams = []
    for word in cat_bigrams_freq_dict.keys():
        if cat_bigrams_freq_dict[word] >= 2: # it it appears at least twice
            weighted_bigrams.append((word, cat_bigrams_freq_dict[word] / all_bigrams_freq_dict[word]))
    # choose only N of the most characteristic bigrams for given category
    weighted_bigrams = sorted(weighted_bigrams, key=lambda tup: tup[1], reverse=True)[:N]
#weighted_bigrams_proportion = int(len(weighted_bigrams) / 5)
    #weighted_bigrams[:len()]
    weighted_bigrams = [tup[0] for tup in weighted_bigrams]
    weighted_bigrams_all.extend(weighted_bigrams)

honorific inscription
6218
500
mile-/leaguestone
2350
500
public legal inscription
3688
500
votive inscription
9314
500
owner/artist inscription
1044
500
building/dedicatory inscription
3832
500
identification inscription
269
269
epitaph
19349
500
elogium
167
167
acclamation
61
61
military diploma
2491
500
list
1362
500
boundary inscription
379
379
label
57
57
defixio
346
346
seat inscription
8
8
private legal inscription
651
500
letter
71
71
prayer
3
3
calendar
228
228
assignation inscription
19
19
adnuntiatio
0
0


In [67]:
len(weighted_bigrams_all)

6608

In [77]:
weighted_bigrams_all = list(set(weighted_bigrams_all))
len(weighted_bigrams_all)

6572

In [33]:
def combine_status_list_and_bigrams(el_status, el_bigrams, el_material):
    # preprocess status:
    try: el_status = eval(el_status)
    except: pass
    if isinstance(el_status, list):
        new_el_status = el_status
    elif isinstance(el_status, str):
        new_el_status = [el_status]
    else:
        new_el_status = []
    new_el_status = [el.replace(" ", "_") for el in new_el_status]
    # preprocess material:
    if el_material is None:
        el_material = []
    else:
        el_material = el_material.partition(": ")[0] #
        el_material = el_material.split(", ")
        el_material = [el.replace(" ", "_").replace("?", "") for el in el_material]
    # combine status, bigrams and material
    new_el = new_el_status + el_bigrams + el_material
    new_el = " ".join(new_el)
    return new_el, new_el_status, el_material

In [34]:
status_bigrams_list = []
status_list = []
material_list = []
for el_status, el_bigrams, el_material in zip(EDH_sampled["status_list"].tolist(), EDH_sampled["bigrams"].tolist(), EDH_sampled["Material"].tolist()):
    new_el, new_status, el_material = combine_status_list_and_bigrams(el_status, el_bigrams, el_material)
    status_bigrams_list.append(new_el)
    status_list.append(new_status)
    material_list.extend(el_material)

In [35]:
materials_freqdist = nltk.FreqDist(material_list).most_common()
len(materials_freqdist)

18

In [36]:
materials_freqdist

[('lapis', 22042),
 ('opus_figlinae', 2247),
 ('aes', 796),
 ('plumbum', 528),
 ('lignum', 289),
 ('argentum', 289),
 ('aurum', 82),
 ('vitrum', 63),
 ('ferrum', 60),
 ('tectorium', 58),
 ('musivum', 27),
 ('steatitis', 11),
 ('os', 10),
 ('cyprum', 6),
 ('gemma', 4),
 ('rupes', 4),
 ('corium', 1),
 ('sucineus', 1)]

In [37]:
N = 20
materials_N = [tup[0] for tup in materials_freqdist[:N]]
len(materials_N)

18

In [38]:
status_list_flat = [el for sublist in status_list for el in sublist]
status_N = [tup[0] for tup in nltk.FreqDist(status_list_flat).most_common(100)] ### we had 228 in total
status_N[:10]

['viri',
 'tituli_sepulcrales',
 'tria_nomina',
 'tituli_sacri',
 'mulieres',
 'milites',
 'nomen_singulare',
 'Augusti/Augustae',
 'tituli_operum',
 'tituli_fabricationis']

In [39]:
el_status = EDH_sampled["status_list"].tolist()[0]
status_list_flat = [el for sublist in status_list for el in sublist]
status_N = [tup[0] for tup in nltk.FreqDist(status_list_flat).most_common(100)] ### we had 228 in total
status_N[:10]

['viri',
 'tituli_sepulcrales',
 'tria_nomina',
 'tituli_sacri',
 'mulieres',
 'milites',
 'nomen_singulare',
 'Augusti/Augustae',
 'tituli_operum',
 'tituli_fabricationis']

In [40]:
len(status_N)

37

In [53]:
vocab = status_N + bigrams_mostfreq[:10000] + materials_N
vocab = list(set(vocab))

In [78]:
def tfidf_vectorizer(data, vocab):
    vocab = list(set(vocab))
    vectorizer = TfidfVectorizer(token_pattern=r"\w+\/?|\_\w+", vocabulary=vocab, lowercase=False)
    X = vectorizer.fit_transform(data)
    X = X.A
    return X

In [79]:
features_statuslist = tfidf_vectorizer(status_bigrams_list, status_N)
features_10000bigrams = tfidf_vectorizer(status_bigrams_list, bigrams_mostfreq[:10000])
features_material = tfidf_vectorizer(status_bigrams_list, materials_N)
features_weightedbigrams = tfidf_vectorizer(status_bigrams_list, weighted_bigrams_all)

In [80]:
print(
    features_statuslist.shape,
    features_10000bigrams.shape,
    features_material.shape,
    features_weightedbigrams.shape)

(46171, 37) (46171, 10000) (46171, 18) (46171, 6572)


In [58]:
with open("../data/large_data/features_statuslist.pickle", "wb") as f:
    pickle.dump(features_statuslist, f, protocol=pickle.HIGHEST_PROTOCOL)
with open("../data/large_data/features_10000bigrams.pickle", "wb") as f:
    pickle.dump(features_10000bigrams, f, protocol=pickle.HIGHEST_PROTOCOL)
with open("../data/large_data/features_material.pickle", "wb") as f:
    pickle.dump(features_material, f, protocol=pickle.HIGHEST_PROTOCOL)
with open("../data/large_data/features_weightedbigrams.pickle", "wb") as f:
    pickle.dump(features_weightedbigrams, f, protocol=pickle.HIGHEST_PROTOCOL)

### A simple test

In [124]:
features = np.hstack([features_statuslist, features_10000bigrams, features_material])

In [134]:
# TRAIN vs TEST
subset_size = 5000 # len(features)

x_train_full, x_test, y_train_full, y_test = train_test_split(features[:subset_size], one_hot_labels[:subset_size], test_size=0.2)

# actual TRAIN vs. internal VALIDATION data
x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, test_size=0.125)

In [135]:
print(
    x_train.shape,
    y_train.shape,
    x_val.shape,
    y_val.shape,
    x_test.shape,
    y_test.shape)

(3500, 10055) (3500, 22) (500, 10055) (500, 22) (1000, 10055) (1000, 22)


In [136]:
# since our classes are highly unbalanced w1 weighted is perhaps the best metric for model performance
def f1_weighted(y_true, y_pred):
    y_true = np.ndarray.argmax(y_true.numpy(), axis=1)
    y_pred = np.ndarray.argmax(y_pred.numpy(), axis=1)
    return f1_score(y_true, y_pred, average="weighted")

In [138]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(len(x_train[0]),)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(22, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=[f1_weighted])

history = model.fit(x_train,
          y_train,
          epochs=15,
          batch_size=256,
          verbose=1,
          validation_data=(x_val, y_val))
results = model.evaluate(x_test, y_test)

Epoch 1/15
 2/14 [===>..........................] - ETA: 0s - loss: 3.0497 - f1_weighted: 0.1451

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/data/ops/dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


14/14 [==============================] - 1s 83ms/step - loss: 2.3160 - f1_weighted: 0.3248 - val_loss: 0.8972 - val_f1_weighted: 0.6847
Epoch 2/15


/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/data/ops/dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


14/14 [==============================] - 1s 78ms/step - loss: 0.7909 - f1_weighted: 0.7394 - val_loss: 0.5980 - val_f1_weighted: 0.7944
Epoch 3/15
14/14 [==============================] - 1s 79ms/step - loss: 0.5169 - f1_weighted: 0.8359 - val_loss: 0.5104 - val_f1_weighted: 0.8260
Epoch 4/15
14/14 [==============================] - 1s 89ms/step - loss: 0.3850 - f1_weighted: 0.8751 - val_loss: 0.4755 - val_f1_weighted: 0.8450
Epoch 5/15
14/14 [==============================] - 1s 80ms/step - loss: 0.3424 - f1_weighted: 0.8949 - val_loss: 0.4556 - val_f1_weighted: 0.8561
Epoch 6/15
14/14 [==============================] - 1s 83ms/step - loss: 0.2892 - f1_weighted: 0.9102 - val_loss: 0.4526 - val_f1_weighted: 0.8600
Epoch 7/15
14/14 [==============================] - 1s 80ms/step - loss: 0.2520 - f1_weighted: 0.9234 - val_loss: 0.4665 - val_f1_weighted: 0.8508
Epoch 8/15
14/14 [==============================] - 1s 79ms/step - loss: 0.2077 - f1_weighted: 0.9338 - val_loss: 0.4640 - val_f1

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/data/ops/dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


32/32 [==============================] - 2s 57ms/step - loss: 0.6721 - f1_weighted: 0.8251


In [139]:
results

[0.6720902323722839, 0.825101912021637]